In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# Initialize empty list to store the URLs
urls = []

# Initialize Selenium webdriver
driver = webdriver.Chrome()  # You need to have Chrome webdriver installed

# Scrape URLs from the forum page
forum_url = "https://www.macanforums.co.uk/viewforum.php?f=20"  # Replace with the actual URL of the forum

# Load the web page
driver.get(forum_url)
time.sleep(2)  # Add a delay to allow the page to load

# Scrape URLs from multiple pages
stop = False

while not stop:
    # Find the HTML elements containing the URLs
    url_elements = driver.find_elements(By.XPATH, '//a[@class="topictitle"]')

    # Iterate through the URLs and add them to the list
    for url_element in url_elements:
        url = url_element.get_attribute('href')
        urls.append(url)

    try:
        # Check if there is a next page button
        next_button = driver.find_element(By.CSS_SELECTOR, 'a[rel="next"]')
        next_button.click()
        time.sleep(2)  # Add a delay to allow the page to load
    except NoSuchElementException:
        stop = True

# Close the webdriver
driver.quit()

# Display the scraped URLs
for url in urls:
    print(url)


https://www.macanforums.co.uk/viewtopic.php?t=13612&sid=364d3b620d37161eec8fc09f7aeba3c4
https://www.macanforums.co.uk/viewtopic.php?t=13607&sid=364d3b620d37161eec8fc09f7aeba3c4
https://www.macanforums.co.uk/viewtopic.php?t=13583&sid=364d3b620d37161eec8fc09f7aeba3c4
https://www.macanforums.co.uk/viewtopic.php?t=13606&sid=364d3b620d37161eec8fc09f7aeba3c4
https://www.macanforums.co.uk/viewtopic.php?t=13604&sid=364d3b620d37161eec8fc09f7aeba3c4
https://www.macanforums.co.uk/viewtopic.php?t=13602&sid=364d3b620d37161eec8fc09f7aeba3c4
https://www.macanforums.co.uk/viewtopic.php?t=13599&sid=364d3b620d37161eec8fc09f7aeba3c4
https://www.macanforums.co.uk/viewtopic.php?t=13601&sid=364d3b620d37161eec8fc09f7aeba3c4
https://www.macanforums.co.uk/viewtopic.php?t=13600&sid=364d3b620d37161eec8fc09f7aeba3c4
https://www.macanforums.co.uk/viewtopic.php?t=12442&sid=364d3b620d37161eec8fc09f7aeba3c4
https://www.macanforums.co.uk/viewtopic.php?t=12870&sid=364d3b620d37161eec8fc09f7aeba3c4
https://www.macanforu

In [3]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from dateutil import parser
from selenium.common.exceptions import NoSuchElementException

# Define a dictionary mapping short month names to full month names
month_mapping = {
    'Jan': 'January',
    'Feb': 'February',
    'Mar': 'March',
    'Apr': 'April',
    'May': 'May',
    'Jun': 'June',
    'Jul': 'July',
    'Aug': 'August',
    'Sep': 'September',
    'Oct': 'October',
    'Nov': 'November',
    'Dec': 'December'
}

# Initialize empty lists to store the data
names = []
comments = []
dates = []

# Initialize Selenium webdriver
driver = webdriver.Chrome()  # You need to have Chrome webdriver installed
wait = WebDriverWait(driver, 10)  # Wait up to 10 seconds for elements to be interactable

# Counter variable to track the number of scraped URLs
url_count = 0

# Iterate through the URLs and scrape the reviews
for url in urls:
    if url_count >= 18000:
        break

    # Load the web page
    driver.get(url)
    time.sleep(2)  # Add a delay to allow the page to load

    # Parse the HTML content of the web page using BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Scrape the reviews from the current page
    while True:
        review_elements = soup.find_all("div", class_=lambda x: x and x.startswith("post has-profile"))

        for review in review_elements:
            name_element = review.find("a", class_="username")
            comment_element = review.find("div", class_="content")
            date_element = review.find("time", datetime=True)

            # Check if the elements exist before accessing their properties
            name = name_element.text.strip() if name_element else ""
            comment = ""

            if comment_element:
                # Exclude the <blockquote> elements with class 'bbblock'
                blockquote_elements = comment_element.find_all("blockquote")
                for blockquote in blockquote_elements:
                    blockquote.decompose()  # Remove the <blockquote> element from the HTML

                # Extract the remaining text content including newline characters
                comment = comment_element.get_text(strip=True).replace("\n", "")

            date = ""
            if date_element:
                date_str = date_element.text.strip()
                date_parts = date_str.split(' ')
                if len(date_parts) >= 3:
                    month = date_parts[1]
                    day = date_parts[2].rstrip(',')
                    year = date_parts[3]
                    date = f"{month} {day}, {year}"
                else:
                    date = ""

            # Replace short form of month names with full month names
            for short_month, full_month in month_mapping.items():
                date = date.replace(short_month, full_month).strip()

            # Append the extracted data to the respective lists
            names.append(name)
            comments.append(comment)
            dates.append(date)

            # Increment the URL count
            url_count += 1

            if url_count >= 18000:
                break

        if url_count >= 18000:
            break

        # Check if there is a next page link
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a[rel='next']")
            next_button.click()
            time.sleep(2)  # Add a delay to allow the page to load
            soup = BeautifulSoup(driver.page_source, 'html.parser')
        except NoSuchElementException:
            break  # Exit the loop if the next button is not found

    if url_count >= 18000:
        break

# Close the webdriver
driver.quit()

# Create a pandas DataFrame from the scraped data
review_data = pd.DataFrame({
    "Name": names,
    "Comment": comments,
    "Date": dates
})

# Display the review data
review_data

,Name,Comment,Date
0,oldpilot,During a regular service at my local OPC (Tonb...,"July 31, 2023"
1,Skyway,Replaced my rear wiper after just 6 months wit...,"July 31, 2023"
2,Bluesnose1812,I usually replace wipers and fill the washer f...,"July 31, 2023"
3,Skyway,Here’s my replacement process for infoviewtopi...,"July 31, 2023"
4,GWL,Part of that £68 is £50 labour.,"July 31, 2023"
...,...,...,...
17995,Wing Commander,I suppose we should be grateful that at least ...,"October 02, 2021"
17996,sd1985,Our first macan was a base on 21s with no Pasm...,"October 02, 2021"
17997,BanZ,"For gen3, on the uk configurator PASM is not c...","October 02, 2021"
17998,Col Lamb,Well that is another reason for avoiding the 2...,"October 02, 2021"


In [4]:
# Save the combined DataFrame as a CSV file
review_data.to_csv('C:/Users/tanay/Downloads/PorscheMacan.csv', index=False, encoding="utf-8-sig")

In [5]:
import time
from datetime import datetime
import pandas as pd
from dateutil import parser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from bs4 import BeautifulSoup
import re

# Initialize empty lists to store the data
names = []
comments = []
dates = []

page = 1  # Starting page number
stop = False

# Initialize Selenium webdriver
driver = webdriver.Chrome()  # You need to have Chrome webdriver installed
wait = WebDriverWait(driver, 10)  # Wait up to 10 seconds for elements to be interactable

while not stop:
    # Calculate the value for the 'i' parameter
    i_value = (page - 1) * 20

    # Load the web page
    url = f"https://www.pistonheads.com/gassing/topic.asp?h=0&f=48&t=1955185&i={i_value}"  # Replace with the actual URL of the reviews page
    driver.get(url)
    time.sleep(2)  # Add a delay to allow the page to load

    try:
        # Wait for the review elements to be present
        review_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".topic-reply")))
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException, Exception):
        # If there's a timeout or element interaction issue, stop the loop
        stop = True
        break

    # Check if there are no more review elements
    if len(review_elements) == 0:
        stop = True
        break

    # Iterate through the review elements and extract the desired information
    for review in review_elements:
        name_element = review.find_element(By.CSS_SELECTOR, "p.author.js-author")
        comment_element = review.find_element(By.CSS_SELECTOR, "div.phml.msg-body")
        date_element = review.find_element(By.CSS_SELECTOR, "span.timestamp")

        # Check if the elements exist before accessing their properties
        if name_element is not None:
            name = name_element.text.strip()
        else:
            name = ""
            
        if comment_element is not None:
        # Exclude div with class 'forumQuote' and extract only the text part
          comment_parts = comment_element.find_elements(By.CSS_SELECTOR, "div:not(.forumQuote)")
          comment = ""
          for part in comment_parts:
              comment += part.text.strip() + " "
          comment = comment.replace("\n", "")
        else:
          comment = ""

        #if comment_element is not None:
          #  comment = comment_element.text.strip().replace("\n", "").replace(f"{name} said:", "")
        #else:
            #comment = ""

        if date_element is not None:
            date_string = re.sub(r'\b(Sunday|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday)\b', '', date_element.text.strip())
            date = parser.parse(date_string).strftime('%B %d, %Y')
        else:
            date = ""

        # Append the extracted data to the respective lists
        names.append(name)
        comments.append(comment)
        dates.append(date)

    # Move to the next page
    page += 1

# Close the webdriver
driver.quit()

# Create a pandas DataFrame from the scraped data
review_dataA = pd.DataFrame({
    "Name": names,
    "Comment": comments,
    "Date": dates
})

# Display the review data
review_dataA

,Name,Comment,Date
0,Gumpclipper,"Good morning all,I am looking at buying my fir...","October 10, 2021"
1,Voodoo Blue,I’m not going to offer an opinion on spec as t...,"October 10, 2021"
2,Gumpclipper,Fantastic info. Thank you! Will take a look at...,"October 10, 2021"
3,TheBMWDriver,Right now 1% is best you can get on the macan....,"October 21, 2021"
4,neilf,My tip would be to get your name on the deposi...,"October 22, 2021"
5,soxboy,I am waiting call back from my OPC about seein...,"October 23, 2021"
6,neilf,Gen 3 is at dealers.My local OPC has a 2.0 and...,"October 25, 2021"


In [6]:
# Initialize empty lists to store the data
names = []
comments = []
dates = []

page = 1  # Starting page number
stop = False

# Initialize Selenium webdriver
driver = webdriver.Chrome()  # You need to have Chrome webdriver installed
wait = WebDriverWait(driver, 10)  # Wait up to 10 seconds for elements to be interactable

while not stop:
    # Calculate the value for the 'i' parameter
    i_value = (page - 1) * 20

    # Load the web page
    url = f"https://www.pistonheads.com/gassing/topic.asp?h=0&f=48&t=1786230&i={i_value}"  # Replace with the actual URL of the reviews page
    driver.get(url)
    time.sleep(2)  # Add a delay to allow the page to load

    try:
        # Wait for the review elements to be present
        review_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".topic-reply")))
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException, Exception):
        # If there's a timeout or element interaction issue, stop the loop
        stop = True
        break

    # Check if there are no more review elements
    if len(review_elements) == 0:
        stop = True
        break

    # Iterate through the review elements and extract the desired information
    for review in review_elements:
        name_element = review.find_element(By.CSS_SELECTOR, "p.author.js-author")
        comment_element = review.find_element(By.CSS_SELECTOR, "div.phml.msg-body")
        date_element = review.find_element(By.CSS_SELECTOR, "span.timestamp")

        # Check if the elements exist before accessing their properties
        if name_element is not None:
            name = name_element.text.strip()
        else:
            name = ""
            
        if comment_element is not None:
        # Exclude div with class 'forumQuote' and extract only the text part
          comment_parts = comment_element.find_elements(By.CSS_SELECTOR, "div:not(.forumQuote)")
          comment = ""
          for part in comment_parts:
              comment += part.text.strip() + " "
          comment = comment.replace("\n", "")
        else:
          comment = ""

        #if comment_element is not None:
          #  comment = comment_element.text.strip().replace("\n", "").replace(f"{name} said:", "")
        #else:
            #comment = ""

        if date_element is not None:
            date_string = re.sub(r'\b(Sunday|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday)\b', '', date_element.text.strip())
            date = parser.parse(date_string).strftime('%B %d, %Y')
        else:
            date = ""

        # Append the extracted data to the respective lists
        names.append(name)
        comments.append(comment)
        dates.append(date)

    # Move to the next page
    page += 1

# Close the webdriver
driver.quit()

# Create a pandas DataFrame from the scraped data
review_dataB = pd.DataFrame({
    "Name": names,
    "Comment": comments,
    "Date": dates
})

# Display the review data
review_dataB

,Name,Comment,Date
0,Nigel Worc's,What ho chapsJust a quick request for feedback...,"January 07, 2019"
1,Cheib,Link appears to be broken!If he’s looking at a...,"January 07, 2019"
2,m88ony,I have had a Macan S Diesel for over 3 years f...,"January 07, 2019"
3,Trevor555,"We had a GTS for a short while.Lovely thing, b...","January 07, 2019"
4,W12GT,Absolutely love my turbo - it does everything ...,"January 08, 2019"
5,notsomadmick,I've had 6 Macans. 2 DS and 4 GTS. I've owned ...,"January 08, 2019"
6,Porsche911R,Nigel Worc's said:What ho chapsJust a quick re...,"January 08, 2019"
7,Koln-RS,Whilst the Macan is not a true Porsche sports ...,"January 08, 2019"
8,Yellow491,Porsche911R said:Nigel Worc's said:What ho cha...,"January 09, 2019"
9,W12GT,W12GT said:Absolutely love my turbo - it does ...,"January 09, 2019"


In [7]:
# Initialize empty lists to store the data
names = []
comments = []
dates = []

page = 1  # Starting page number
stop = False

# Initialize Selenium webdriver
driver = webdriver.Chrome()  # You need to have Chrome webdriver installed
wait = WebDriverWait(driver, 10)  # Wait up to 10 seconds for elements to be interactable

while not stop:
    # Calculate the value for the 'i' parameter
    i_value = (page - 1) * 20

    # Load the web page
    url = f"https://www.pistonheads.com/gassing/topic.asp?h=0&f=23&t=1943744&i={i_value}"  # Replace with the actual URL of the reviews page
    driver.get(url)
    time.sleep(2)  # Add a delay to allow the page to load

    try:
        # Wait for the review elements to be present
        review_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".topic-reply")))
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException, Exception):
        # If there's a timeout or element interaction issue, stop the loop
        stop = True
        break

    # Check if there are no more review elements
    if len(review_elements) == 0:
        stop = True
        break

    # Iterate through the review elements and extract the desired information
    for review in review_elements:
        name_element = review.find_element(By.CSS_SELECTOR, "p.author.js-author")
        comment_element = review.find_element(By.CSS_SELECTOR, "div.phml.msg-body")
        date_element = review.find_element(By.CSS_SELECTOR, "span.timestamp")

        # Check if the elements exist before accessing their properties
        if name_element is not None:
            name = name_element.text.strip()
        else:
            name = ""
            
        if comment_element is not None:
        # Exclude div with class 'forumQuote' and extract only the text part
          comment_parts = comment_element.find_elements(By.CSS_SELECTOR, "div:not(.forumQuote)")
          comment = ""
          for part in comment_parts:
              comment += part.text.strip() + " "
          comment = comment.replace("\n", "")
        else:
          comment = ""

        #if comment_element is not None:
          #  comment = comment_element.text.strip().replace("\n", "").replace(f"{name} said:", "")
        #else:
            #comment = ""

        if date_element is not None:
            date_string = re.sub(r'\b(Sunday|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday)\b', '', date_element.text.strip())
            date = parser.parse(date_string).strftime('%B %d, %Y')
        else:
            date = ""

        # Append the extracted data to the respective lists
        names.append(name)
        comments.append(comment)
        dates.append(date)

    # Move to the next page
    page += 1

# Close the webdriver
driver.quit()

# Create a pandas DataFrame from the scraped data
review_dataC = pd.DataFrame({
    "Name": names,
    "Comment": comments,
    "Date": dates
})

# Display the review data
review_dataC

,Name,Comment,Date
0,pb8g09,With an unlimited budget and only space for 1 ...,"July 24, 2021"
1,Johnnytheboy,Friend of mine (my mechanic) had one until rec...,"July 24, 2021"
2,Rostyle,Ok not a huge fan of Macan anyway as no rear l...,"July 24, 2021"
3,sidewinder500,"And it isn't a Porsche anyway, it's an Audi...","July 24, 2021"
4,cerb4.5lee,"I've always wanted to try one of these, and fr...","July 24, 2021"
5,Buster73,sidewinder500 said:And it isn't a Porsche anyw...,"July 24, 2021"
6,sidewinder500,Buster73 said:sidewinder500 said:And it isn't ...,"July 24, 2021"
7,houseofbroon,"Really nice cars both to look at and to drive,...","July 24, 2021"
8,sidesauce,sidewinder500 said:Buster73 said:sidewinder500...,"July 24, 2021"
9,mrclav,houseofbroon said:It's a long time since Porch...,"July 24, 2021"


In [8]:
# Initialize empty lists to store the data
names = []
comments = []
dates = []

page = 1  # Starting page number
stop = False

# Initialize Selenium webdriver
driver = webdriver.Chrome()  # You need to have Chrome webdriver installed
wait = WebDriverWait(driver, 10)  # Wait up to 10 seconds for elements to be interactable

while not stop:
    # Calculate the value for the 'i' parameter
    i_value = (page - 1) * 20

    # Load the web page
    url = f"https://www.pistonheads.com/gassing/topic.asp?h=0&f=23&t=1998280&i={i_value}"  # Replace with the actual URL of the reviews page
    driver.get(url)
    time.sleep(2)  # Add a delay to allow the page to load

    try:
        # Wait for the review elements to be present
        review_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".topic-reply")))
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException, Exception):
        # If there's a timeout or element interaction issue, stop the loop
        stop = True
        break

    # Check if there are no more review elements
    if len(review_elements) == 0:
        stop = True
        break

    # Iterate through the review elements and extract the desired information
    for review in review_elements:
        name_element = review.find_element(By.CSS_SELECTOR, "p.author.js-author")
        comment_element = review.find_element(By.CSS_SELECTOR, "div.phml.msg-body")
        date_element = review.find_element(By.CSS_SELECTOR, "span.timestamp")

        # Check if the elements exist before accessing their properties
        if name_element is not None:
            name = name_element.text.strip()
        else:
            name = ""
            
        if comment_element is not None:
        # Exclude div with class 'forumQuote' and extract only the text part
          comment_parts = comment_element.find_elements(By.CSS_SELECTOR, "div:not(.forumQuote)")
          comment = ""
          for part in comment_parts:
              comment += part.text.strip() + " "
          comment = comment.replace("\n", "")
        else:
          comment = ""

        #if comment_element is not None:
          #  comment = comment_element.text.strip().replace("\n", "").replace(f"{name} said:", "")
        #else:
            #comment = ""

        if date_element is not None:
            date_string = re.sub(r'\b(Sunday|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday)\b', '', date_element.text.strip())
            date = parser.parse(date_string).strftime('%B %d, %Y')
        else:
            date = ""

        # Append the extracted data to the respective lists
        names.append(name)
        comments.append(comment)
        dates.append(date)

    # Move to the next page
    page += 1

# Close the webdriver
driver.quit()

# Create a pandas DataFrame from the scraped data
review_dataD = pd.DataFrame({
    "Name": names,
    "Comment": comments,
    "Date": dates
})

# Display the review data
review_dataD

,Name,Comment,Date
0,PistonTim,"So the car cant stand up to a British summer, ...","August 11, 2022"
1,CoolHands,I remember the first time I saw a Porsche Caye...,"August 11, 2022"
2,Wab1974uk,Macan S is the sweet spot.A YouTuber compared ...,"August 11, 2022"
3,thelostboy,I went from Diesel Macan on steel springs to G...,"August 11, 2022"
4,130R,I had a new base Macan as a courtesy car a cou...,"August 11, 2022"
...,...,...,...
57,as7920,swanseaboydan said:No plans to track mine . .I...,"August 15, 2022"
58,swanseaboydan,Let us know how you get on - we’ve been offere...,"August 15, 2022"
59,as7920,swanseaboydan said:Let us know how you get on ...,"August 15, 2022"
60,AMV93,swanseaboydan said:Let us know how you get on ...,"August 16, 2022"


In [9]:
# Initialize empty lists to store the data
names = []
comments = []
dates = []

page = 1  # Starting page number
stop = False

# Initialize Selenium webdriver
driver = webdriver.Chrome()  # You need to have Chrome webdriver installed
wait = WebDriverWait(driver, 10)  # Wait up to 10 seconds for elements to be interactable

while not stop:
    # Calculate the value for the 'i' parameter
    i_value = (page - 1) * 20

    # Load the web page
    url = f"https://www.pistonheads.com/gassing/topic.asp?h=0&f=23&t=1959957&i={i_value}"  # Replace with the actual URL of the reviews page
    driver.get(url)
    time.sleep(2)  # Add a delay to allow the page to load

    try:
        # Wait for the review elements to be present
        review_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".topic-reply")))
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException, Exception):
        # If there's a timeout or element interaction issue, stop the loop
        stop = True
        break

    # Check if there are no more review elements
    if len(review_elements) == 0:
        stop = True
        break

    # Iterate through the review elements and extract the desired information
    for review in review_elements:
        name_element = review.find_element(By.CSS_SELECTOR, "p.author.js-author")
        comment_element = review.find_element(By.CSS_SELECTOR, "div.phml.msg-body")
        date_element = review.find_element(By.CSS_SELECTOR, "span.timestamp")

        # Check if the elements exist before accessing their properties
        if name_element is not None:
            name = name_element.text.strip()
        else:
            name = ""
            
        if comment_element is not None:
        # Exclude div with class 'forumQuote' and extract only the text part
          comment_parts = comment_element.find_elements(By.CSS_SELECTOR, "div:not(.forumQuote)")
          comment = ""
          for part in comment_parts:
              comment += part.text.strip() + " "
          comment = comment.replace("\n", "")
        else:
          comment = ""

        #if comment_element is not None:
          #  comment = comment_element.text.strip().replace("\n", "").replace(f"{name} said:", "")
        #else:
            #comment = ""

        if date_element is not None:
            date_string = re.sub(r'\b(Sunday|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday)\b', '', date_element.text.strip())
            date = parser.parse(date_string).strftime('%B %d, %Y')
        else:
            date = ""

        # Append the extracted data to the respective lists
        names.append(name)
        comments.append(comment)
        dates.append(date)

    # Move to the next page
    page += 1

# Close the webdriver
driver.quit()

# Create a pandas DataFrame from the scraped data
review_dataE = pd.DataFrame({
    "Name": names,
    "Comment": comments,
    "Date": dates
})

# Display the review data
review_dataE

,Name,Comment,Date
0,CDP,The thick end of 50 grand for a four cylinder ...,"November 13, 2021"
1,MyV10BarksAndBites,Straight BS!!!,"November 13, 2021"
2,CDP,MyV10BarksAndBites said:Straight BS!!!Me or th...,"November 13, 2021"
3,Oso,"A clever argument, but even the most myopicall...","November 13, 2021"
4,Oso,"Hey PH, is it the same deal with the press off...","November 13, 2021"
...,...,...,...
135,Cobnapint,The only problem for me (as a Cayenne diesel d...,"November 16, 2021"
136,PaulJC84,We got one of the 2.0 versions as a courtesy n...,"November 16, 2021"
137,cerb4.5lee,Cobnapint said:The only problem for me (as a C...,"November 16, 2021"
138,PaulJC84,I always see comments about how it doesnt feel...,"November 16, 2021"


In [10]:
# Initialize empty lists to store the data
names = []
comments = []
dates = []

page = 1  # Starting page number
stop = False

# Initialize Selenium webdriver
driver = webdriver.Chrome()  # You need to have Chrome webdriver installed
wait = WebDriverWait(driver, 10)  # Wait up to 10 seconds for elements to be interactable

while not stop:
    # Calculate the value for the 'i' parameter
    i_value = (page - 1) * 20

    # Load the web page
    url = f"https://www.pistonheads.com/gassing/topic.asp?h=0&f=48&t=1606742&i={i_value}"  # Replace with the actual URL of the reviews page
    driver.get(url)
    time.sleep(2)  # Add a delay to allow the page to load

    try:
        # Wait for the review elements to be present
        review_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".topic-reply")))
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException, Exception):
        # If there's a timeout or element interaction issue, stop the loop
        stop = True
        break

    # Check if there are no more review elements
    if len(review_elements) == 0:
        stop = True
        break

    # Iterate through the review elements and extract the desired information
    for review in review_elements:
        name_element = review.find_element(By.CSS_SELECTOR, "p.author.js-author")
        comment_element = review.find_element(By.CSS_SELECTOR, "div.phml.msg-body")
        date_element = review.find_element(By.CSS_SELECTOR, "span.timestamp")

        # Check if the elements exist before accessing their properties
        if name_element is not None:
            name = name_element.text.strip()
        else:
            name = ""
            
        if comment_element is not None:
        # Exclude div with class 'forumQuote' and extract only the text part
          comment_parts = comment_element.find_elements(By.CSS_SELECTOR, "div:not(.forumQuote)")
          comment = ""
          for part in comment_parts:
              comment += part.text.strip() + " "
          comment = comment.replace("\n", "")
        else:
          comment = ""

        #if comment_element is not None:
          #  comment = comment_element.text.strip().replace("\n", "").replace(f"{name} said:", "")
        #else:
            #comment = ""

        if date_element is not None:
            date_string = re.sub(r'\b(Sunday|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday)\b', '', date_element.text.strip())
            date = parser.parse(date_string).strftime('%B %d, %Y')
        else:
            date = ""

        # Append the extracted data to the respective lists
        names.append(name)
        comments.append(comment)
        dates.append(date)

    # Move to the next page
    page += 1

# Close the webdriver
driver.quit()

# Create a pandas DataFrame from the scraped data
review_dataF = pd.DataFrame({
    "Name": names,
    "Comment": comments,
    "Date": dates
})

# Display the review data
review_dataF

,Name,Comment,Date
0,Avalyn,I had my Boxster S 987.2 in for it's annual MO...,"July 08, 2016"
1,POORCARDEALER,Avalyn said:I had my Boxster S 987.2 in for it...,"July 08, 2016"
2,EricE,Technically the Macan is heavily reworked Q5. ...,"July 08, 2016"
3,Avalyn,POORCARDEALER said:Bought wifey a Cayenne dies...,"July 08, 2016"
4,MDL111,Avalyn said:POORCARDEALER said:Bought wifey a ...,"July 08, 2016"
...,...,...,...
64,breadvan,Fish said:Main point though is there are thous...,"September 27, 2016"
65,squirejo,langlord said:Squirejo - What are you comparin...,"September 28, 2016"
66,Fish,We hardly ever see Macans up here in the midla...,"September 28, 2016"
67,Buster73,Fish said:We hardly ever see Macans up here in...,"September 28, 2016"


In [11]:
# Initialize empty lists to store the data
names = []
comments = []
dates = []

page = 1  # Starting page number
stop = False

# Initialize Selenium webdriver
driver = webdriver.Chrome()  # You need to have Chrome webdriver installed
wait = WebDriverWait(driver, 10)  # Wait up to 10 seconds for elements to be interactable

while not stop:
    # Calculate the value for the 'i' parameter
    i_value = (page - 1) * 20

    # Load the web page
    url = f"https://www.pistonheads.com/gassing/topic.asp?h=0&f=23&t=1949478&i={i_value}"  # Replace with the actual URL of the reviews page
    driver.get(url)
    time.sleep(2)  # Add a delay to allow the page to load

    try:
        # Wait for the review elements to be present
        review_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".topic-reply")))
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException, Exception):
        # If there's a timeout or element interaction issue, stop the loop
        stop = True
        break

    # Check if there are no more review elements
    if len(review_elements) == 0:
        stop = True
        break

    # Iterate through the review elements and extract the desired information
    for review in review_elements:
        name_element = review.find_element(By.CSS_SELECTOR, "p.author.js-author")
        comment_element = review.find_element(By.CSS_SELECTOR, "div.phml.msg-body")
        date_element = review.find_element(By.CSS_SELECTOR, "span.timestamp")

        # Check if the elements exist before accessing their properties
        if name_element is not None:
            name = name_element.text.strip()
        else:
            name = ""
            
        if comment_element is not None:
        # Exclude div with class 'forumQuote' and extract only the text part
          comment_parts = comment_element.find_elements(By.CSS_SELECTOR, "div:not(.forumQuote)")
          comment = ""
          for part in comment_parts:
              comment += part.text.strip() + " "
          comment = comment.replace("\n", "")
        else:
          comment = ""

        #if comment_element is not None:
          #  comment = comment_element.text.strip().replace("\n", "").replace(f"{name} said:", "")
        #else:
            #comment = ""

        if date_element is not None:
            date_string = re.sub(r'\b(Sunday|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday)\b', '', date_element.text.strip())
            date = parser.parse(date_string).strftime('%B %d, %Y')
        else:
            date = ""

        # Append the extracted data to the respective lists
        names.append(name)
        comments.append(comment)
        dates.append(date)

    # Move to the next page
    page += 1

# Close the webdriver
driver.quit()

# Create a pandas DataFrame from the scraped data
review_dataG = pd.DataFrame({
    "Name": names,
    "Comment": comments,
    "Date": dates
})

# Display the review data
review_dataG

,Name,Comment,Date
0,Honeywell,Hideous gin palace for the Cheshire wives brig...,"September 01, 2021"
1,Deerfoot,I’m always fascinated by Porsche option prices...,"September 01, 2021"
2,ddom,Honeywell said:Hideous gin palace for the Ches...,"September 01, 2021"
3,Kipsrs,"Have to agree with ddom. . No garish options, ...","September 01, 2021"
4,Turini,I struggle to not add £10k minimum whenever I ...,"September 01, 2021"
...,...,...,...
107,fatboy b,,"September 05, 2021"
108,Adam.,voram said:Only if you're desperate to impress...,"September 05, 2021"
109,MortyC137,MortyC137 said:There's no such thing as Porsch...,"September 07, 2021"
110,px1980,I bought my 2018 Macan Turbo Perf Pack in Dece...,"September 07, 2021"


In [12]:
# Initialize empty lists to store the data
names = []
comments = []
dates = []

page = 1  # Starting page number
stop = False

# Initialize Selenium webdriver
driver = webdriver.Chrome()  # You need to have Chrome webdriver installed
wait = WebDriverWait(driver, 10)  # Wait up to 10 seconds for elements to be interactable

while not stop:
    # Calculate the value for the 'i' parameter
    i_value = (page - 1) * 20

    # Load the web page
    url = f"https://www.pistonheads.com/gassing/topic.asp?h=0&f=48&t=1580475&i={i_value}"  # Replace with the actual URL of the reviews page
    driver.get(url)
    time.sleep(2)  # Add a delay to allow the page to load

    try:
        # Wait for the review elements to be present
        review_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".topic-reply")))
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException, Exception):
        # If there's a timeout or element interaction issue, stop the loop
        stop = True
        break

    # Check if there are no more review elements
    if len(review_elements) == 0:
        stop = True
        break

    # Iterate through the review elements and extract the desired information
    for review in review_elements:
        name_element = review.find_element(By.CSS_SELECTOR, "p.author.js-author")
        comment_element = review.find_element(By.CSS_SELECTOR, "div.phml.msg-body")
        date_element = review.find_element(By.CSS_SELECTOR, "span.timestamp")

        # Check if the elements exist before accessing their properties
        if name_element is not None:
            name = name_element.text.strip()
        else:
            name = ""
            
        if comment_element is not None:
        # Exclude div with class 'forumQuote' and extract only the text part
          comment_parts = comment_element.find_elements(By.CSS_SELECTOR, "div:not(.forumQuote)")
          comment = ""
          for part in comment_parts:
              comment += part.text.strip() + " "
          comment = comment.replace("\n", "")
        else:
          comment = ""

        #if comment_element is not None:
          #  comment = comment_element.text.strip().replace("\n", "").replace(f"{name} said:", "")
        #else:
            #comment = ""

        if date_element is not None:
            date_string = re.sub(r'\b(Sunday|Monday|Tuesday|Wednesday|Thursday|Friday|Saturday)\b', '', date_element.text.strip())
            date = parser.parse(date_string).strftime('%B %d, %Y')
        else:
            date = ""

        # Append the extracted data to the respective lists
        names.append(name)
        comments.append(comment)
        dates.append(date)

    # Move to the next page
    page += 1

# Close the webdriver
driver.quit()

# Create a pandas DataFrame from the scraped data
review_dataH = pd.DataFrame({
    "Name": names,
    "Comment": comments,
    "Date": dates
})

# Display the review data
review_dataH

,Name,Comment,Date
0,Koln-RS,Don't know much about the Macan. Always resent...,"March 13, 2016"
1,tree7777,The residuals are league above Cayenne. I have...,"March 13, 2016"
2,atom111,I've a Petrol turbo covered over 32000 miles n...,"March 13, 2016"
3,ZeroH,If you actually want a family car then imo its...,"March 13, 2016"
4,ORD,I intended to test drive one recently but deci...,"March 15, 2016"
5,EricE,"Relatively speaking the Macan is Porsche's ""ho...","March 15, 2016"
6,Dr S,I can only recommend a test drive. The Macan i...,"March 15, 2016"
7,hopeydaze,How does it compare with an Evoque? I have onl...,"March 15, 2016"
8,Geneve,I have had a Cayenne 3.0d for 5 years - family...,"March 15, 2016"
9,ORD,Diesel Porsche. Jesus wept.,"March 15, 2016"


In [15]:
# Combine the DataFrames into a single DataFrame
combined_data = pd.concat([review_dataA, review_dataB, review_dataC, review_dataD, review_dataE, review_dataF, review_dataG, review_dataH])

# Save the combined DataFrame as a CSV file
combined_data.to_csv('C:/Users/tanay/Downloads/Macan_pistonheads.csv', index=False, encoding="utf-8-sig")